In [1]:
import numpy as np
import pandas as pd
import torch.nn as nn
import torch
from types import SimpleNamespace
from torch.nn.utils.rnn import pad_sequence
import torch.optim as optim
import random
import torch.nn.functional as F
import wandb
from utilities import *

In [2]:

train,valid,test=load_data(data_path,lang)

add_start_end(train)
add_start_end(valid)
add_start_end(test)

train_src_chars,train_target_chars=get_unique_chars(train)
valid_src_chars,valid_target_chars=get_unique_chars(valid)
test_src_chars,test_target_chars=get_unique_chars(test)
train_target_chars.add('*') # extra char to handle unknowns in valid and test data.
    
src_char_idx,src_idx_char=get_char_map(train_src_chars)
target_char_idx,target_idx_char=get_char_map(train_target_chars)

encoder_vocab_size=len(src_char_idx)+1 # one extra for padding
decoder_vocab_size=len(target_char_idx)+1 # one extra for padding

max_seq_length=train[0].apply(lambda x:len(x)).max()
max_target_length=train[1].apply(lambda x:len(x)).max()

train_src_int,train_target_int=vectorize(train,src_char_idx,target_char_idx,max_seq_length)
valid_src_int,valid_target_int=vectorize(valid,src_char_idx,target_char_idx,max_seq_length)
test_src_int,test_target_int=vectorize(test,src_char_idx,target_char_idx,max_seq_length)



def main():
    wandb.init()
    config=wandb.config 
    run_name=f'Attention -Yes Cell-{config.cell_type} Hidden-{config.hidden_size} Embedding-{config.embedding_size} Bidir-{config.bidirectional} Dropout -{config.dropout} EL-{config.encoder_num_layers} DL-{config.decoder_num_layers}'
    wandb.run.name=run_name
    config.encoder_vocab_size=encoder_vocab_size
    config.decoder_vocab_size=decoder_vocab_size
    config.max_seq_length=max_seq_length

    model=AttentionSeq2Seq(config).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss(ignore_index=0)
    
    for epoch in range(config.epochs):
        train_loss=0
        train_acc=0
        model.train()
        batch_no=0
        for data in get_batch(train_src_int,train_target_int,config.batch_size):
#             print(batch_no)
            batch_no+=1
            x=data[0]
            y=data[1]
            x=x.to(torch.int64).T
            y=y.to(torch.int64).T
            outputs,_=model.forward(x,y)
            output=outputs.reshape(-1,outputs.shape[2])
            target=y.reshape(-1)
            optimizer.zero_grad()
            target=target-1
            target[target<0]=0
            loss = criterion(output, target)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
            optimizer.step()
            train_loss+=loss.item()*config.batch_size

            batch_acc=cal_acc(outputs,y)
            train_acc+=batch_acc
        train_loss/=len(train_src_int)
        train_acc/=batch_no 
        model.eval()

        valid_loss=0
        valid_acc=0
        batch_no=0
        with torch.no_grad():
            for data in get_batch(valid_src_int,valid_target_int,config.batch_size):
                batch_no+=1
                x=data[0]
                y=data[1]
                x=x.to(torch.int64).T
                y=y.to(torch.int64).T
                outputs,_=model.forward(x,y,prediction=True)
                output=outputs.reshape(-1,outputs.shape[2])
                target=y.reshape(-1)
                target=target-1
                target[target<0]=0
                loss = criterion(output, target)
                valid_loss+=loss.item()*config.batch_size
                valid_acc+=cal_acc(outputs,y)
            valid_loss/=len(valid_src_int)
            valid_acc/=batch_no
        print(f'Epoch: {epoch+1} Train Loss: {train_loss:.4f} Valid Loss: {valid_loss:.4f} Train Acc: {train_acc:.4f}  Valid Acc: {valid_acc:.4f}')
        wandb.log({'train accuracy':train_acc,'train loss':train_loss,'valid accuracy':valid_acc,'valid loss':valid_loss})
    wandb.finish()
    return model



sweep_config= {
    'method': 'bayes',
    'name': 'Attention Sweep',
    'metric': {
        'goal': 'maximize', 
        'name': 'valid accuracy'
      },
    "parameters":
    {
        'hidden_size': {"values":[32,64,128,256,512]},
        'batch_size': {"values":[64,128,256]},
        'encoder_num_layers': {"values":[1,2,3]},
        'decoder_num_layers': {"values":[1,2,3]},
        'embedding_size': {"values":[128,256]},
        'dropout': {"values":[0,0.2,0.3]},
        'epochs':{"values":[15]},
        'cell_type':{"values":['LSTM',"GRU","RNN"]},
        'bidirectional':{"values":["Yes","No"]}
    }
}

sweep_id = wandb.sweep(sweep_config, project='DL_3_Assign')
wandb.agent(sweep_id, main, count=75)
wandb.finish()   

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: 84o4oww1
Sweep URL: https://wandb.ai/cs22m080/DL_3_Assign/sweeps/84o4oww1


wandb: Agent Starting Run: 408g5t70 with config:
wandb: 	batch_size: 64
wandb: 	bidirectional: Yes
wandb: 	cell_type: LSTM
wandb: 	decoder_num_layers: 3
wandb: 	dropout: 0
wandb: 	embedding_size: 256
wandb: 	encoder_num_layers: 3
wandb: 	epochs: 15
wandb: 	hidden_size: 256
wandb: Currently logged in as: cs22m080. Use `wandb login --relogin` to force relogin


Epoch: 1 Train Loss: 1.9537 Valid Loss: 1.3351 Train Acc: 0.0294  Valid Acc: 0.1709
Epoch: 2 Train Loss: 0.8840 Valid Loss: 1.1059 Train Acc: 0.2596  Valid Acc: 0.3464
Epoch: 3 Train Loss: 0.6877 Valid Loss: 1.0231 Train Acc: 0.4180  Valid Acc: 0.4102
Epoch: 4 Train Loss: 0.6072 Valid Loss: 0.9999 Train Acc: 0.5029  Valid Acc: 0.4543
Epoch: 5 Train Loss: 0.5587 Valid Loss: 0.9860 Train Acc: 0.5575  Valid Acc: 0.4668
Epoch: 6 Train Loss: 0.5235 Valid Loss: 0.9977 Train Acc: 0.6037  Valid Acc: 0.4824
Epoch: 7 Train Loss: 0.4958 Valid Loss: 1.0082 Train Acc: 0.6434  Valid Acc: 0.4888
Epoch: 8 Train Loss: 0.4733 Valid Loss: 1.0039 Train Acc: 0.6761  Valid Acc: 0.5012
Epoch: 9 Train Loss: 0.4591 Valid Loss: 1.0435 Train Acc: 0.7014  Valid Acc: 0.4871
Epoch: 10 Train Loss: 0.4436 Valid Loss: 1.0474 Train Acc: 0.7309  Valid Acc: 0.5166
Epoch: 11 Train Loss: 0.4299 Valid Loss: 1.0495 Train Acc: 0.7598  Valid Acc: 0.5151
Epoch: 12 Train Loss: 0.4198 Valid Loss: 1.0745 Train Acc: 0.7791  Valid A

train accuracy,▁▃▄▅▆▆▆▇▇▇▇████
train loss,█▃▂▂▂▂▁▁▁▁▁▁▁▁▁
valid accuracy,▁▄▆▇▇▇▇▇▇██████
valid loss,█▃▂▁▁▁▁▁▂▂▂▃▄▄▄
train accuracy,0.83127
train loss,0.39743
valid accuracy,0.5144
valid loss,1.13224


...

train accuracy,▁▄▅▆▆▇▇▇▇██████
train loss,█▃▂▂▂▁▁▁▁▁▁▁▁▁▁
valid accuracy,▁▃▅▅▇▇██▇████▇▆
valid loss,█▅▃▃▃▁▄▄▃▅▄▆▄▄▇
train accuracy,0.76201
train loss,0.43075
valid accuracy,0.52808
valid loss,1.04575


wandb: Agent Starting Run: tp5cenez with config:
wandb: 	batch_size: 256
wandb: 	bidirectional: Yes
wandb: 	cell_type: RNN
wandb: 	decoder_num_layers: 1
wandb: 	dropout: 0.2
wandb: 	embedding_size: 128
wandb: 	encoder_num_layers: 2
wandb: 	epochs: 15
wandb: 	hidden_size: 512


wandb: WARNING Config item 'dropout' was locked by 'sweep' (ignored update).


Epoch: 1 Train Loss: 1.5447 Valid Loss: 1.2025 Train Acc: 0.1374  Valid Acc: 0.2769
Epoch: 2 Train Loss: 0.7080 Valid Loss: 1.0320 Train Acc: 0.4110  Valid Acc: 0.4399
Epoch: 3 Train Loss: 0.6193 Valid Loss: 0.9741 Train Acc: 0.4978  Valid Acc: 0.4800
Epoch: 4 Train Loss: 0.5718 Valid Loss: 0.9628 Train Acc: 0.5425  Valid Acc: 0.4868
Epoch: 5 Train Loss: 0.5441 Valid Loss: 0.9681 Train Acc: 0.5719  Valid Acc: 0.5059
Epoch: 6 Train Loss: 0.5321 Valid Loss: 0.9770 Train Acc: 0.5941  Valid Acc: 0.5039
Epoch: 7 Train Loss: 0.5092 Valid Loss: 1.0418 Train Acc: 0.6195  Valid Acc: 0.4797
Epoch: 8 Train Loss: 0.4906 Valid Loss: 0.9508 Train Acc: 0.6399  Valid Acc: 0.5256
Epoch: 9 Train Loss: 0.4880 Valid Loss: 0.9461 Train Acc: 0.6500  Valid Acc: 0.5303
Epoch: 10 Train Loss: 0.4767 Valid Loss: 0.9431 Train Acc: 0.6674  Valid Acc: 0.5547
Epoch: 11 Train Loss: 0.4699 Valid Loss: 0.9656 Train Acc: 0.6796  Valid Acc: 0.5466
Epoch: 12 Train Loss: 0.4711 Valid Loss: 0.9585 Train Acc: 0.6805  Valid A

train accuracy,▁▄▅▆▆▇▇▇▇██████
train loss,█▃▂▂▂▁▁▁▁▁▁▁▁▁▁
valid accuracy,▁▅▆▆▇▇▆▇▇██████
valid loss,█▃▂▂▂▂▄▁▁▁▂▁▂▂▁
train accuracy,0.70045
train loss,0.45547
valid accuracy,0.53955
valid loss,0.95953
